In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE93698"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE93698"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE93698.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE93698.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE93698.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles in systemic sclerosis tenosynovial biopsies"
!Series_summary	"Systemic sclerosis is a connective tissue disease affecting skin and internal organs, characterized by a triad of inflammation, vasculopathy and progressive fibrosis, due to deposition of mainly type I collagen. Out of the intricate mechanisms involved in the pathogenesis of the disease, evidence indicates that TGFbeta signaling plays a central role in mediating the effects of several pro-fibrotic effectors. In addition, TGFbeta is induced by hypoxia in cultured fibroblasts, an observation suggesting a role for this cytokine in linking vasculopathy and fibrosis in the disease. Not surprisingly, TGFbeta and Wnt signaling are among the most prevalent pathways found in global gene expression studies performed on systemic sclerosis skin biopsies. In this perspective, modulation of TGFbeta activity remains a top therapeutic target in systemic sclerosis drug developme

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# The dataset appears to be gene expression data, not miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# trait (Osteoarthritis)
# From sample characteristics, I can see 'disease state' in row 1 has 'Osteoarthritis' as a value
trait_row = 1

# age
# Age is available in row 2
age_row = 2

# gender
# Gender is available in row 3
gender_row = 3

# 2.2 Data Type Conversion

# trait conversion function
def convert_trait(value):
    """Convert trait value to binary (0 for no, 1 for yes)"""
    if pd.isna(value):
        return None
    
    value = value.lower()
    if 'disease state:' in value:
        value = value.split('disease state:')[1].strip()
    
    # Check if the current value is for the trait we're interested in (Osteoarthritis)
    if trait.lower() in value.lower():
        return 1
    else:
        return 0

# age conversion function
def convert_age(value):
    """Convert age value to continuous"""
    if pd.isna(value):
        return None
    
    # Extract the age value after the colon
    if 'age:' in value:
        try:
            age_val = value.split('age:')[1].strip()
            return float(age_val)
        except:
            return None
    return None

# gender conversion function
def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    
    value = value.lower()
    if 'gender:' in value:
        gender_val = value.split('gender:')[1].strip()
        if gender_val == 'f':
            return 0
        elif gender_val == 'm':
            return 1
    return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering of dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract and process clinical features
if trait_row is not None and 'clinical_data' in globals():
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the clinical features to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM2460692': [0.0, nan, nan], 'GSM2460693': [0.0, nan, nan], 'GSM2460694': [0.0, nan, nan], 'GSM2460695': [0.0, nan, nan], 'GSM2460696': [0.0, nan, nan], 'GSM2460733': [1.0, 71.0, 0.0], 'GSM2460734': [1.0, 73.0, 0.0], 'GSM2460735': [1.0, 65.0, 0.0], 'GSM2460736': [1.0, 51.0, 0.0], 'GSM2460737': [1.0, 56.0, 1.0], 'GSM2460738': [0.0, 52.0, 1.0], 'GSM2460739': [0.0, 42.0, 1.0], 'GSM2460740': [0.0, 43.0, 0.0], 'GSM2460741': [0.0, 69.0, 1.0], 'GSM2460742': [0.0, 55.0, 1.0], 'GSM2460743': [0.0, 62.0, 1.0], 'GSM2460744': [0.0, 37.0, 0.0], 'GSM2460745': [0.0, 38.0, 0.0], 'GSM2460746': [0.0, 19.0, 0.0], 'GSM2460747': [0.0, 40.0, 0.0], 'GSM2460748': [0.0, 31.0, 0.0], 'GSM2460749': [0.0, 45.0, 1.0], 'GSM2460750': [0.0, 55.0, 1.0], 'GSM2460751': [0.0, 72.0, 1.0], 'GSM2460752': [0.0, 42.0, 1.0], 'GSM2460753': [0.0, 72.0, 0.0], 'GSM2460754': [0.0, 28.0, 0.0], 'GSM2460755': [0.0, 39.0, 0.0], 'GSM2460756': [0.0, 47.0, 1.0], 'GSM2460757': [0.0, 21.0, 1.0]}
Clin

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE93698/GSE93698_series_matrix.txt.gz


Gene data shape: (54675, 30)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing gene identifiers
# The identifiers ('1007_s_at', '1053_at', etc.) appear to be Affymetrix probe IDs 
# rather than standard human gene symbols (which would look like BRCA1, TP53, etc.)
# These probe IDs need to be mapped to human gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation that contain probe IDs and gene symbols
# From the gene annotation preview, we can see:
# - 'ID' column contains the probe IDs (same as in gene_data.index)
# - 'Gene Symbol' column contains the gene symbols

# 2. Get gene mapping dataframe with the probe ID and gene symbol columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Sample of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted gene expression data shape: {gene_data.shape}")
print("First 5 genes and their expression values:")
print(gene_data.head())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (45782, 2)
Sample of mapping data:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Converted gene expression data shape: (21278, 30)
First 5 genes and their expression values:
          GSM2460692  GSM2460693  GSM2460694  GSM2460695  GSM2460696  \
Gene                                                                   
A1BG        0.003618   -0.234991   -0.027722    0.366942    0.245110   
A1BG-AS1   -0.247791   -0.692595   -0.120352    0.292251    0.000284   
A1CF       -0.106579   -1.037735   -0.575760   -0.453761   -0.176319   
A2M        -1.141026   -0.091595    0.088030   -0.307626    0.513546   
A2M-AS1    -1.137488    0.337934    0.031662   -0.573664    0.683518   

          GSM2460733  GSM2460734  GSM2460735  GSM2460736  GSM2460737  ...  \
Gene                                                                  ... 

Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE93698.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {gene_data.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Load clinical features data if needed
# First check if we already have the clinical features from previous steps
if not os.path.exists(out_clinical_data_file):
    # If not, extract it again from the matrix file
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Extract clinical features using the conversion functions from Step 2
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
else:
    # Load previously saved clinical data
    selected_clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)

print("Clinical data shape:", selected_clinical_df.shape)
print("Clinical data preview:", preview_df(selected_clinical_df))

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Evaluate bias in trait and demographic features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation
note = "Dataset contains gene expression data from synovial and tenosynovial biopsies, including patients with Osteoarthritis."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (19845, 30)


Normalized gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE93698.csv
Clinical data shape: (3, 30)
Clinical data preview: {'GSM2460692': [0.0, nan, nan], 'GSM2460693': [0.0, nan, nan], 'GSM2460694': [0.0, nan, nan], 'GSM2460695': [0.0, nan, nan], 'GSM2460696': [0.0, nan, nan], 'GSM2460733': [1.0, 71.0, 0.0], 'GSM2460734': [1.0, 73.0, 0.0], 'GSM2460735': [1.0, 65.0, 0.0], 'GSM2460736': [1.0, 51.0, 0.0], 'GSM2460737': [1.0, 56.0, 1.0], 'GSM2460738': [0.0, 52.0, 1.0], 'GSM2460739': [0.0, 42.0, 1.0], 'GSM2460740': [0.0, 43.0, 0.0], 'GSM2460741': [0.0, 69.0, 1.0], 'GSM2460742': [0.0, 55.0, 1.0], 'GSM2460743': [0.0, 62.0, 1.0], 'GSM2460744': [0.0, 37.0, 0.0], 'GSM2460745': [0.0, 38.0, 0.0], 'GSM2460746': [0.0, 19.0, 0.0], 'GSM2460747': [0.0, 40.0, 0.0], 'GSM2460748': [0.0, 31.0, 0.0], 'GSM2460749': [0.0, 45.0, 1.0], 'GSM2460750': [0.0, 55.0, 1.0], 'GSM2460751': [0.0, 72.0, 1.0], 'GSM2460752': [0.0, 42.0, 1.0], 'GSM2460753': [0.0, 72.0, 0.0], 'GSM2460754': [0.

Linked data shape after handling missing values: (30, 19848)
For the feature 'Osteoarthritis', the least common label is '1.0' with 5 occurrences. This represents 16.67% of the dataset.
The distribution of the feature 'Osteoarthritis' in this dataset is fine.

Quartiles for 'Age':
  25%: 40.5
  50% (Median): 49.0
  75%: 55.75
Min: 19.0
Max: 73.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 12 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Osteoarthritis/GSE93698.csv
